<a href="https://colab.research.google.com/github/rafaelhamer/DSWP/blob/master/Labdata_Churn_parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
for dirname, _, filenames in os.walk('/gdrive/My Drive/Python'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
def mostra_missing_value(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

## Lendo os dados de Treino e Teste

In [28]:
df_train = pd.read_csv('/gdrive/My Drive/Python/train.csv', index_col='id')
df_test  = pd.read_csv('/gdrive/My Drive/Python/test.csv', index_col='id')

In [ ]:
df_train.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
id,,,,,,,,,,,,,,,,,,,,
4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1
6368,Male,0,No,No,70.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,Bank transfer (automatic),48.40,3442.8,0
5830,Male,0,Yes,Yes,31.0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,59.95,1848.8,0
4476,Female,0,No,No,3.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,19.55,61.05,0
1508,Male,0,No,No,54.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),19.65,1008.7,0


In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.dtypes

**TotalCharges**. Clientes novos estão com um espaço no valor total pago. Foi trocado esse valor para zero e convertido o campo para float.

In [29]:
df_train['TotalCharges'] = df_train['TotalCharges'].str.replace(' ','0.0').astype(np.float64)
df_test['TotalCharges'] = df_test['TotalCharges'].str.replace(' ','0.0').astype(np.float64)

**tenure** é o número de meses do assinante. Em caso de NaN, está sendo assumido como sendo o (total pago)/(mensalidade)

In [30]:
df_test['tenure'] = df_test['tenure'].fillna(df_test['TotalCharges'] / df_test['MonthlyCharges'])
df_train['tenure'] = df_train['tenure'].fillna(df_train['TotalCharges'] / df_train['MonthlyCharges'])
df_train.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
id,,,,,,,,,,,,,,,,,,,,
4030,Female,0,No,No,56.000000,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.10,0
6731,Male,0,Yes,Yes,16.928753,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
6479,Female,0,Yes,No,60.000000,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,0
6861,Female,0,No,No,37.000000,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
3266,Male,0,Yes,Yes,29.000000,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,1


**TotalCharges**. Desnecessário porque se torna redundante com **tenure** e MonthlyCharges.

In [ ]:
df_train.drop('TotalCharges', axis=1, inplace=True)
df_test.drop('TotalCharges', axis=1, inplace=True)
df_train.head(5)

In [33]:
df_train.drop('gender', axis=1, inplace=True)
df_test.drop('gender', axis=1, inplace=True)
df_train.head(5)

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
id,,,,,,,,,,,,,,,,,,
4030,0,No,No,56.000000,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,0
6731,0,Yes,Yes,16.928753,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,0
6479,0,Yes,No,60.000000,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,0
6861,0,No,No,37.000000,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,1
3266,0,Yes,Yes,29.000000,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1


In [18]:
df_train[['OnlineBackup','DeviceProtection','Churn']].value_counts()

OnlineBackup         DeviceProtection     Churn
No internet service  No internet service  0        1137
Yes                  Yes                  0         860
No                   No                   0         854
                                          1         736
Yes                  No                   0         649
No                   Yes                  0         639
                                          1         259
Yes                  No                   1         241
                     Yes                  1         172
No internet service  No internet service  1          87
dtype: int64

In [21]:
df_train[['OnlineSecurity','TechSupport','Churn']].value_counts()

OnlineSecurity       TechSupport          Churn
No internet service  No internet service  0        1137
No                   No                   0        1024
                                          1        1008
Yes                  Yes                  0         808
                     No                   0         605
No                   Yes                  0         565
                                          1         166
Yes                  No                   1         156
No internet service  No internet service  1          87
Yes                  Yes                  1          78
dtype: int64

In [ ]:
df_train[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']].value_counts()

**Dependents** com valores NaN são preenchidos com "No"

In [34]:
df_train['Dependents'].fillna('No', inplace=True)
df_test['Dependents'].fillna('No', inplace=True)

In [35]:
#df_train.dropna(subset=['PaymentMethod'],inplace=True)
df_train['PaymentMethod'].fillna('Nao identificado', inplace=True)
df_test['PaymentMethod'].fillna('Nao identificado', inplace=True)

In [25]:
df_train['PaymentMethod'].value_counts()

Electronic check             1887
Mailed check                 1272
Bank transfer (automatic)    1212
Credit card (automatic)      1164
Nao identificado               99
Name: PaymentMethod, dtype: int64

In [36]:
mostra_missing_value(df_train)

                 Total  Percentual
Churn                0         0.0
MonthlyCharges       0         0.0
Partner              0         0.0
Dependents           0         0.0
tenure               0         0.0
PhoneService         0         0.0
MultipleLines        0         0.0
InternetService      0         0.0
OnlineSecurity       0         0.0
OnlineBackup         0         0.0


In [37]:
mostra_missing_value(df_test)

                 Total  Percentual
MonthlyCharges       0         0.0
OnlineSecurity       0         0.0
Partner              0         0.0
Dependents           0         0.0
tenure               0         0.0
PhoneService         0         0.0
MultipleLines        0         0.0
InternetService      0         0.0
OnlineBackup         0         0.0
PaymentMethod        0         0.0


In [ ]:
train_backup = df_train.copy()
test_backup = df_test.copy()

In [ ]:
df_train = train_backup.copy()
df_test = test_backup.copy()

In [38]:
df_train

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
id,,,,,,,,,,,,,,,,,,
4030,0,No,No,56.000000,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,Nao identificado,45.05,0
6731,0,Yes,Yes,16.928753,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,0
6479,0,Yes,No,60.000000,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,0
6861,0,No,No,37.000000,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,1
3266,0,Yes,Yes,29.000000,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,0,No,No,43.000000,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,Electronic check,44.15,0
604,0,No,No,45.128394,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,No,Electronic check,108.65,0
4803,0,No,No,28.000000,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,Bank transfer (automatic),54.65,0


## Transformando variáveis categóricas com `LabelEncoder`

In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
for cat_var in df_train.select_dtypes(include='O').columns:
    le = LabelEncoder()
    le.fit(df_train[cat_var])
    df_train[cat_var + '_num'] = le.transform(df_train[cat_var])
    df_train.drop(cat_var, axis=1, inplace=True)
    df_test[cat_var + '_num'] = le.transform(df_test[cat_var])
    df_test.drop(cat_var, axis=1, inplace=True)

In [41]:
df_train.head()

,SeniorCitizen,tenure,MonthlyCharges,Churn,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,
4030,0,56.000000,45.05,0,0,0,0,1,0,2,2,2,2,0,0,2,0,4
6731,0,16.928753,19.65,0,1,1,1,0,2,1,1,1,1,1,1,1,0,3
6479,0,60.000000,110.80,0,1,0,1,2,1,0,2,2,2,2,2,2,1,1
6861,0,37.000000,101.90,1,0,0,1,2,1,0,2,2,0,2,2,0,1,0
3266,0,29.000000,70.75,1,1,1,1,0,1,0,0,0,0,0,0,0,1,2


In [42]:
df_test.head()

,SeniorCitizen,tenure,MonthlyCharges,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,
5027,0,23.0,20.00,1,1,1,0,2,1,1,1,1,1,1,1,1,1
1733,1,61.0,99.00,1,1,1,2,1,0,2,0,0,2,2,1,0,4
5384,0,36.0,84.75,0,0,1,2,1,0,0,0,0,0,2,0,1,2
6554,0,61.0,61.45,1,1,0,1,0,0,2,2,2,2,2,2,1,0
364,0,47.0,20.55,0,0,1,0,2,1,1,1,1,1,1,2,1,3


## Eliminando variáveis

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
df_train_limpo = df_train.copy()
df_test_limpo = df_test.copy()

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df_train_limpo)

array([[5.60000000e+01, 4.50500000e+01, 2.56010000e+03, ...,
        2.00000000e+00, 0.00000000e+00, 4.00000000e+00],
       [1.69287532e+01, 1.96500000e+01, 3.32650000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 3.00000000e+00],
       [6.00000000e+01, 1.10800000e+02, 6.64070000e+03, ...,
        2.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [2.80000000e+01, 5.46500000e+01, 1.51750000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.30000000e+01, 2.52500000e+01, 1.57305000e+03, ...,
        2.00000000e+00, 0.00000000e+00, 3.00000000e+00],
       [3.80000000e+01, 9.50000000e+01, 3.60560000e+03, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
df_train_limpo

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_num,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,,,
4030,0,56.000000,45.05,2560.10,0,0,0,0,0,1,0,2,2,2,2,0,0,2,0,4
6731,0,16.928753,19.65,332.65,0,1,1,1,1,0,2,1,1,1,1,1,1,1,0,3
6479,0,60.000000,110.80,6640.70,0,0,1,0,1,2,1,0,2,2,2,2,2,2,1,1
6861,0,37.000000,101.90,3545.35,1,0,0,0,1,2,1,0,2,2,0,2,2,0,1,0
3266,0,29.000000,70.75,1974.80,1,1,1,1,1,0,1,0,0,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,0,43.000000,44.15,1931.30,0,1,0,0,0,1,0,0,0,2,2,0,2,1,0,2
604,0,45.128394,108.65,4903.20,0,1,0,0,1,2,1,0,2,2,2,2,2,2,0,2
4803,0,28.000000,54.65,1517.50,0,0,0,0,1,0,0,0,2,0,2,0,0,0,0,0


In [ ]:
sel

VarianceThreshold(threshold=0.15999999999999998)

## Salvando o dataframe tratado

In [43]:
df_train.to_csv('/gdrive/My Drive/Python/treino_limpo2.csv')
df_test.to_csv('/gdrive/My Drive/Python/teste_limpo2.csv')